<a href="https://colab.research.google.com/github/katduecker/nma_dl_bearmin/blob/main/bear_min_data_prep_class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
class ecog_preproc:

  def __init__(self, fname, l_cutoff, h_cutoff, sfreq, trange, bslrange, freqs, subjid, expid, ncyc):
 """fname: filename
    *_cutoff: frequencies for high and low pass filter
    trange: time vector for each epoch
    bslrange: baseline interval (used for baseline correction/relative baseline)
    sfreq: sampling rate
    freqs: frequencies of interest
    subjid: subject ID (0-6)
    expid: experiment (0: passive viewing faces + houses , 1: viewing faces, houses, noisy images)
    ncyc: number of cycles for morlet tfr
 
 """
    self.fname = fname
    self.l_cutoff = l_cutoff
    self.h_cutoff = h_cutoff
    self.sfreq = sfreq
    self.trange = trange
    self.bslrange = bslrange
    self.freqs = freqs
    self.subjid = subjid

  def load_raw(self):
    #@title load data
    alldat = np.load(self.fname, allow_pickle=True)['dat']
    # select just one of the recordings here. 
    dat1 = alldat[subjid][expid]

    return dat1

  def run_filter(self, dat1):
    ch_names = [f'ecog{n:03}' for n in range(1, dat1['V'].shape[1]+1)] 
    ch_types='ecog'
    info = mne.create_info(ch_names, ch_types=ch_types, sfreq=self.sfreq)
    info['description'] = 'Example pat no 1'

    V = dat1['V'].astype('float32')
    nt, nchan = V.shape
    nstim = len(dat1['t_on'])

    # V.transpose() ?
    raw = mne.io.RawArray(V.transpose(), info, first_samp=0, copy='auto' )
    raw_filtered = raw.filter(l_freq=self.l_cutoff, h_freq=self.h_cutoff, method='fir', fir_design='firwin', filter_length='auto', fir_window='hamming', phase='zero', 
              l_trans_bandwidth='auto', h_trans_bandwidth='auto')
    V_filtered = raw_filtered['data'][0].transpose()

    return V_filtered, info

  def make_epochs(self, dat1, V_filtered, info):

    nt, nchan = V_filtered.shape
    nstim = len(dat1['t_on'])

    ts = dat1['t_on'][:,np.newaxis] + self.trange
    V_epochs = np.reshape(V_filtered[ts, :], (nstim, 600, nchan))

    V_epochs = np.transpose(V_epochs, [0, 2, 1])

    # extract events
    events = np.zeros((dat1['t_on'].shape[0],3), dtype=int)
    events[:,0] = dat1['t_on'] 
    events[:,1] = dat1['t_off'] 
    eve_ids = dat1['stim_id'].copy()

    eve_ids[eve_ids <= 50] = 1
    eve_ids[eve_ids > 50] = 2
    events[:,2] = eve_ids
    event_dict = dict(houses=1, faces=2)
    epochs = mne.EpochsArray(V_epochs, info, tmin=-0.2, events=events,
                                   event_id=event_dict, baseline=bslrange)
    
    return epochs

  def mor_tfr(self, epochs):

    powface = tfr_morlet(epochs['faces'], freqs=self.freqs, n_cycles=self.ncyc, return_itc=False, average=False)
    powhouse = tfr_morlet(epochs['houses'], freqs=self.freqs, n_cycles=self.ncyc, return_itc=False, average=False)

    powface.apply_baseline(bslrange, mode='logratio')
    powhouse.apply_baseline(bslrange, mode='logratio')

    return powface, powhouse

IndentationError: ignored